A lot of the code here was inspired from Paul Mooney's [notebook on TF-DF](https://www.kaggle.com/code/paultimothymooney/getting-started-with-tensorflow-decision-forests). Make sure to check that out as well.

# Introduction

A learning algorithm trains a machine learning model on a training dataset. The parameters of a learning algorithm–called "hyper-parameters"–control how the model is trained and impact its quality. Therefore, finding the best hyper-parameters is an important stage of modeling.

Automated tuning algorithms work by generating and evaluating a large number of hyper-parameter values. Each of those iterations is called a "trial". The evaluation of a trial is expensive as it requires to train a new model each time. At the end of the tuning, the hyper-parameter with the best evaluation is used.

To demonstrate automated hyper-parameter tuning in TF-DF we'll be working with the Tabular Playground Series Feb 2021 Kaggle Dataset. It is a tabular dataset with 300,000 rows and 26 columns in training (93.66 MiB .CSV training dataset + 58.85 MiB .CSV test set) that is suitable for training algorithms to solve regression problems.

We'll be predicting a continuous target based on a number of feature columns given in the data. All of the feature columns, cat0 - cat9 are categorical, and the feature columns cont0 - cont13 are continuous.

By studying this tutorial you will learn how to quickly and automatically tune hyper-parameters of a GradientBoostedTrees model to perform a regression task using tabular data.

# Installing TensorFlow Decision Forests

In [ ]:
# Display only the messages with ERROR, CRITICAL log levels
!pip install tensorflow_decision_forests -U -qq

# Importing libraries

In [ ]:
# Import Python packages
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_decision_forests as tfdf
print("TensorFlow Decision Forests v" + tfdf.__version__)

# Helper functions

In [ ]:
# Define helper functions for plotting training evaluation curves

def plot_tfdf_model_training_curves(model):
    # This function was adapted from the following tutorial:
    # https://www.tensorflow.org/decision_forests/tutorials/beginner_colab
    logs = model.make_inspector().training_logs()
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    # Plot RMSE vs number of trees
    plt.plot([log.num_trees for log in logs], [log.evaluation.rmse for log in logs])
    plt.xlabel("Number of trees")
    plt.ylabel("RMSE (out-of-bag)")
    plt.show()

In [ ]:
# print list of all data and files attached to this notebook
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load the dataset and convert it in a tf.Dataset

The dataset contains a mix of numerical (e.g. cont0 - cont13) and categorical (e.g. cat0 - cat9) features. TF-DF supports all these feature types natively (differently than NN based models), therefore there is no need for preprocessing in the form of one-hot encoding or normalization. Also by default the task is set to Classification in TF-DF, we'll change that to Regression.

In [ ]:
# load to pandas dataframe (for data exploration)
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')

# load to tensorflow dataset (for model training)
train_tfds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="target", task=tfdf.keras.Task.REGRESSION)
test_tfds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, task=tfdf.keras.Task.REGRESSION)

# Exploratory data analysis

In [ ]:
# print column names
print(train_df.columns)

In [ ]:
# preview first few rows of data
train_df.head(10)

In [ ]:
# print basic summary statistics
train_df.describe()

In [ ]:
# check for missing values
sns.heatmap(train_df.isnull(), cbar=False)

# Automatic hyper-parameter tuning

Hyper-paramter tuning is enabled by specifying the tuner constructor argument of the model. The tuner object contains all the configuration of the tuner (search space, optimizer, trial and objective).

In [ ]:
# This code block was adapted from the following tutorial:
# https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab

# Configure the tuner.

# Create a Random Search tuner with 10 trials for demo purpose, tune with
# atleast 50 trials in practice.
tuner = tfdf.tuner.RandomSearch(num_trials=10)

# Define the search space.
#
# Adding more parameters generaly improve the quality of the model, but make
# the tuning last longer.

tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

# Some hyper-parameters are only valid for specific values of other
# hyper-parameters. For example, the "max_depth" parameter is mostly useful when
# "growing_strategy=LOCAL" while "max_num_nodes" is better suited when
# "growing_strategy=BEST_FIRST_GLOBAL".

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

# merge=True indicates that the parameter (here "growing_strategy") is already
# defined, and that new values are added to it.
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

# Regression tasks currently does not support hessian optimization in TF-DF
# https://github.com/tensorflow/decision-forests/issues/116
# tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])
tuner.choice("split_axis", ["AXIS_ALIGNED"])

oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

In addition to having a default set of hyper-parameters, TF-DF also provides you with a list of additional hyper-parameter choices to consider.

In [ ]:
print(tfdf.keras.GradientBoostedTreesModel.predefined_hyperparameters())

However, here we'll not be using the default hyper-parameter choices and tune them to get the best results instead. Also by default the task is set to Classification in TF-DF, we'll change that to Regression.

In [ ]:
# Tune the model. Notice the `tuner=tuner`.
gb_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner, task=tfdf.keras.Task.REGRESSION)
gb_model.fit(x=train_tfds, verbose=2)

In [ ]:
# Display the tuning logs.
tuning_logs = gb_model.make_inspector().tuning_logs()
tuning_logs.head()

In [ ]:
# Best hyper-parameters.
tuning_logs[tuning_logs.best].iloc[0]

# Plot the model

We plot the evoluation of the best score during the training and then the tuning.

In [ ]:
plot_tfdf_model_training_curves(gb_model)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(tuning_logs["score"], label="current trial")
plt.plot(tuning_logs["score"].cummax(), label="best trial")
plt.xlabel("Tuning step")
plt.ylabel("Tuning score")
plt.legend()
plt.show()

# Evaluate the model

For this dataset, submissions are scored on the root mean squared error. Hence we evaluate the model on that metrics.

In [ ]:
gb_model.compile(metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
inspector = gb_model.make_inspector()
inspector.evaluation()

In [ ]:
gb_model.evaluate(train_tfds)

In [ ]:
print("Model type:", inspector.model_type())
print("Objective:", inspector.objective())
print("Evaluation:", inspector.evaluation())

Variable importances generally indicates how much a variable contributes to the model predictions or quality. Variable importance SUM_SCORE is sum of the split scores using a specific feature. The larger, the most important.

In [ ]:
inspector.variable_importances()["SUM_SCORE"]

In [ ]:
gb_model.summary()

# Make submission

In [ ]:
sample_submission_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')
sample_submission_df['target'] = gb_model.predict(test_tfds)
sample_submission_df.to_csv('/kaggle/working/submission.csv', index=False)
sample_submission_df.head()